#for colab

In [1]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1

Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (41.0.1)


In [2]:
!pip uninstall pyglet
!pip install --user pyglet==1.3.2

Uninstalling pyglet-1.3.2:
  Would remove:
    /root/.local/lib/python3.6/site-packages/pyglet-1.3.2.dist-info/*
    /root/.local/lib/python3.6/site-packages/pyglet/*
Proceed (y/n)? y
  Successfully uninstalled pyglet-1.3.2
  Using cached https://files.pythonhosted.org/packages/1c/fc/dad5eaaab68f0c21e2f906a94ddb98175662cc5a654eee404d59554ce0fa/pyglet-1.3.2-py2.py3-none-any.whl


In [0]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

In [4]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

W0726 14:16:51.504336 140411038521216 abstractdisplay.py:151] xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1041'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1041'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [0]:
# env = wrap_env(gym.make("MsPacman-v0"))
# show_video()

#main

In [7]:
!pip install chainerrl

In [0]:
import gym
from gym import wrappers
import numpy as np
import time
import chainer
import chainer.functions as F
import chainer.links as L
import chainerrl

In [0]:
from tqdm import tqdm

In [0]:
class QFunction(chainer.Chain):
    def __init__(self, obs_size, n_actions, n_hidden_channels=50):
        super(QFunction, self).__init__()
        with self.init_scope():
            self.l0 = L.Linear(obs_size, n_hidden_channels)
            self.l1 = L.Linear(n_hidden_channels, n_hidden_channels)
            self.l2 = L.Linear(n_hidden_channels, n_actions)
    def __call__(self, x, test=False):
        h = F.tanh(self.l0(x))
        h = F.tanh(self.l1(h))
        return chainerrl.action_value.DiscreteActionValue(self.l2(h))

In [0]:
env = wrap_env(gym.make('CartPole-v0'))

In [0]:
gamma = 0.99
alpha = 0.5
max_number_of_steps = 200
num_episodes = 300

In [0]:
q_func = QFunction(env.observation_space.shape[0], env.action_space.n)
optimizer = chainer.optimizers.Adam(eps=1e-2)
optimizer.setup(q_func)
explorer = chainerrl.explorers.LinearDecayEpsilonGreedy(
    start_epsilon = 1.0,
    end_epsilon = 0.1,
    decay_steps = num_episodes,
    random_action_func = env.action_space.sample
)
replay_buffer = chainerrl.replay_buffer.ReplayBuffer(capacity=10**6)
phi = lambda x: x.astype(np.float32, copy=False)

In [0]:
agent = chainerrl.agents.DQN(
    q_func, optimizer, replay_buffer, gamma, explorer,
    replay_start_size = 500,
    update_interval = 1,
    target_update_interval = 100,
    phi = phi
)

In [15]:
for episode in tqdm(range(num_episodes)):
    videos = []
    observation = env.reset()
    done = False
    reward = 0
    R = 0
    for t in range(max_number_of_steps):
        action = agent.act_and_train(observation, reward)
        observation, reward, done, info = env.step(action)
        R += reward
        if done:
            break
    agent.stop_episode_and_train(observation, reward, done)
    if episode % 10 == 0:
        print('episode: ', episode, 'R: ', R, 'statistics', agent.get_statistics())
    if episode % 100 == 0:
            env.render()
            show_video()
            time.sleep(10)

  0%|          | 0/300 [00:00<?, ?it/s]

episode:  0 R:  16.0 statistics [('average_q', 0.003486876554258312), ('average_loss', 0), ('n_updates', 0)]


  4%|▎         | 11/300 [00:14<07:45,  1.61s/it]

episode:  10 R:  85.0 statistics [('average_q', 0.01798434120616137), ('average_loss', 0.15052950054620712), ('n_updates', 46)]


  7%|▋         | 21/300 [00:18<01:50,  2.52it/s]

...

 94%|█████████▎| 281/300 [04:55<00:22,  1.16s/it]

episode:  280 R:  194.0 statistics [('average_q', 99.33049866766868), ('average_loss', 0.5437317017597709), ('n_updates', 41371)]


 97%|█████████▋| 291/300 [05:05<00:10,  1.12s/it]

episode:  290 R:  200.0 statistics [('average_q', 99.44118359535923), ('average_loss', 0.5929584648825299), ('n_updates', 43081)]


100%|██████████| 300/300 [05:15<00:00,  1.11s/it]
